## Monitoring memory in python

In [ ]:
import psutil

In [ ]:
# psutil.virtual_memory?

In [ ]:
vmem = psutil.virtual_memory()
vmem

In [ ]:
total_mem_bytes = vmem.total
total_mem_bytes

In [ ]:
avail_mem_bytes = vmem.available
avail_mem_bytes

## File probing

In [ ]:
import gzip

def probe_file(fpath, lines_to_probe=100):
    with gzip.open(fpath, 'rt') as f:
        for i, line in enumerate(f):
            print(line)
            if i==lines_to_probe:
                return

## Reading lines

In [ ]:
from ezenum import StringEnum
LINETYPE = StringEnum(['Time', 'User', 'Content', 'Other'])

In [ ]:
def interpret_line(decoded_line):
    try:
        char1 = decoded_line[1]
        if char1 == '\t':
            char0 = decoded_line[0]
            if char0 == 'T':
                return LINETYPE.Time, decoded_line[2:-1]
            elif char0 == 'U':
                return LINETYPE.User, decoded_line[21:-1]
            elif char0 == 'W':
                return LINETYPE.Content, decoded_line[2:-1]
            else:
                return LINETYPE.Other, decoded_line[2:-1]
        else:
            return LINETYPE.Other, ''
    except IndexError:
        return LINETYPE.Other, ''

In [ ]:
import gzip
from time import time

NO_CONTENT = 'No Post Title'
user_to_tweets_str = {}
# user_to_tweets_count = {}
monitor_line_freq = 1000000
min_avail_mem_bytes = 500 * 1000000
report_template = (
    '{:.2f} min running | {} lines processed | ~ {} tweets processed |'
    ' {} tpm | {} available memory'
)


def merge_user_tweets_in_file(fpath, line_limit):
    most_recent_user = None
    starting_available_mem = psutil.virtual_memory().available
    start_time = time()
    with gzip.open(fpath, 'rt') as textf:
        for i, line in enumerate(textf):
            try:
#                 print(line)
#                 print(interpret_line(line))
                ltype, lcontent = interpret_line(line)
                if ltype == LINETYPE.User:
                    most_recent_user = lcontent
                elif ltype == LINETYPE.Content and lcontent != NO_CONTENT:
                    user_to_tweets_str[most_recent_user] = user_to_tweets_str.get(
                        most_recent_user, '') + ' ' + lcontent
#                     user_to_tweets_count[most_recent_user] = user_to_tweets_count.get(most_recent_user, 0) + 1
            except Exception as e:
                print(line)
                print(interpret_line(line))
                raise e
            if i % monitor_line_freq == 0:
                av_mem = psutil.virtual_memory().available
                seconds_running = time() - start_time
                report = report_template.format(
                    seconds_running / 60,
                    i,
                    i / 4,
                    (i / 4) / (seconds_running / 60),
                    av_mem,
                )
                print(report, end='\r')
                if av_mem < min_avail_mem_bytes:
                    return
#             if i >= line_limit:
#                 break

In [ ]:
source1_fpath = '/home/shaypalachy/data/twitter7/tweets2009-06.txt.gz'

In [ ]:
sample_fpath = '/home/shaypalachy/data/twitter7/twitter7_sample.txt.gz'

In [ ]:
merge_user_tweets_in_file(fpath1, 10000)

In [ ]:
# user_to_tweets_str

In [ ]:
len(user_to_tweets_str)

In [ ]:
user_to_tweets_str['poluakerford']

In [ ]:
# user_to_tweets_count

In [ ]:
import sys

In [ ]:
sys.getsizeof(user_to_tweets_str)

In [ ]:
sys.getsizeof(user_to_tweets_count)

## Run finished code

In [ ]:
from twikwak17.sample import sample_twitter7

In [ ]:
sample_twitter7(num_tweets=50000, source_fpath=source1_fpath)

In [1]:
from twikwak17.phases.phase1 import merge_user_tweets_in_file

In [2]:
sample_fpath = '/home/shaypalachy/data/twitter7/twitter7_sample.txt.gz'

In [3]:
merge_user_tweets_in_file(sample_fpath)

Merging tweets by user in /home/shaypalachy/data/twitter7/twitter7_sample.txt.gz.
Monitor line frequency is 1000000 and min allowed memory (MB) is 500.


In [4]:
probe_file('/home/shaypalachy/data/twitter7/twitter7_sample_p1dump_0.txt.gz', 100)

NameError: name 'probe_file' is not defined